# NLP in tensorflow

In [6]:
import os 
import pathlib
import tensorflow as tf 

In [ ]:
# get data
data_dir = pathlib.Path("../../all-data/nlp-getting-started")
if not data_dir.exists():
    tf.keras.utils.get_file(
        'nlp_getting_started.zip',
        origin="https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip", 
        cache_dir='.',
        # cache_subdir='data', 
        extract=True
    )

In [3]:
# get helper functions
tf.keras.utils.get_file(
    'helper_functions.py', 
    origin="https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py", 
    cache_dir='.'
)

24576/10246 [=======================================================================] - 0s 0us/step


'./datasets/helper_functions.py'

In [15]:
# visualize the data
import pandas as pd

train_df = pd.read_csv("../../all-data/nlp-getting-started/train.csv")
test_df = pd.read_csv("../../all-data/nlp-getting-started/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
# get helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [16]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [17]:
# Test head
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [18]:
# How many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [19]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [22]:
# Visualize random training samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
to whomever is hijacking my wifi hotspot. I have a very specific skill set. I will create a character and perform a one-man show about you

---

Target: 0 (not real disaster)
Text:
I tried making a chocolate and peanut butter lava cake using my #shakeology protein shake mix and aÛ_ https://t.co/APoD4EIVBa

---

Target: 1 (real disaster)
Text:
oc73x mhtw4fnet

Officials: Alabama home quarantined over possible Ebola case - Washington Post

---

Target: 1 (real disaster)
Text:
Rainstorm Destroys 600 Houses in Yobe State http://t.co/nU0D3uANNZ

---

Target: 0 (not real disaster)
Text:
@FunkyLilShack @mariaf30 I want a full on bitch slapping guns blazing cake throwing Charles showdown!! Now THAT will be worth the wait ????

---



In [25]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(), 
    train_df_shuffled["target"].to_numpy(), 
    test_size=0.1, 
    random_state=42
)

In [26]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [27]:
# view few training sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,